In [1]:
%load_ext autoreload

%autoreload 2
import os


In [2]:

from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import T5Config
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from transformers import set_seed


# In[3]:


from datetime import datetime
import argparse
import os
import sys
import numpy as np

sys.path.append("./hf_transformers/")



In [3]:
import torch

from data_reader import GetDataAsPython
from prepare_data import create_data
from prepare_data import create_dataset
from prepare_data import extract_warning_types
from utils import boolean_string
from utils import get_current_time


# In[34]:


import torch

from data_reader import GetDataAsPython
from prepare_data import create_data
from prepare_data import create_dataset
from prepare_data import extract_warning_types
from utils import boolean_string
from utils import get_current_time
import csv

start_all = datetime.now()

# In[6]:

In [4]:
local = True
base_model = 'training/t5-small_repo-based_21-01-2022_10-29-42/checkpoint-16440'

if local:
    storage_directory = './storage/'
    load_model = f'./{storage_directory}/{base_model}'
    batch_size = 16
else:
    storage_directory = '/scratch/arminz/'
    batch_size = 64
    load_model = f'/{storage_directory}/{base_model}'

# In[7]:


import random


# In[8]:


exec_number = random.randint(0, 1000)
exec_number

336

In [5]:

# parser = argparse.ArgumentParser()
# parser.add_argument("-r", "--repo", type=str, default='/data/all/data/oroinc/platform')
# parser.add_argument("-p", "--percent", type=float, default=1)

# args = parser.parse_args()
repo = "/data/all/data/qooxdoo/qooxdoo" #args.repo
sample_percent = 1#args.percent

print('start:', repo, sample_percent)

lr = 4e-3
ws = 300
wd = 0.4
print('best arguments', lr, wd, ws)




# In[35]:


name='curr'
name


# In[36]:


# Read and prepare data
data = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_repo_specific_final.json")
data_eslint = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_eslint_final.json")
data += data_eslint


len(data)

start: /data/all/data/qooxdoo/qooxdoo 1
best arguments 0.004 0.4 300


104804

In [6]:

all_warning_types = extract_warning_types(data)


# In[39]:


(train_inputs, train_labels, val_inputs, val_labels, test_inputs, test_labels, train_info, val_info, test_info, ) = \
    create_data(data, all_warning_types, include_warning=True, design='repo-based-included', select_repo=repo)


# In[40]:


tokenizer = T5Tokenizer.from_pretrained(load_model)


# In[41]:


len(train_inputs)


# In[42]:



# Create dataset required by pytorch
samples = int(sample_percent * len(train_inputs))
train_dataset = create_dataset(train_inputs[:samples], train_labels[:samples], tokenizer, pad_truncate=True, max_length=128)
val_dataset = create_dataset(val_inputs, val_labels, tokenizer, pad_truncate=True)

print(f'amount of data that is being used for fine-tuning (train) : {len(train_dataset)} == {samples} ({sample_percent})')
print(f'amount of data that is being used for fine-tuning (validation): {len(val_dataset)} (full)')
print(f'amount of data that will be probably being used for testing: {sum([len(x) for x in test_inputs.values()])} (full)')

# In[61]:

splitting by : repo-based-included
train size: 322
val size: 110
test size: 115
amount of data that is being used for fine-tuning (train) : 322 == 322 (1)
amount of data that is being used for fine-tuning (validation): 110 (full)
amount of data that will be probably being used for testing: 115 (full)


/home/armin/TFix/env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [7]:
type(train_dataset)

prepare_data.BugFixDataset

In [8]:
# from transformers import AutoTokenizer, DataCollatorWithPadding

# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)


# def tokenize_function(example):
#     return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
# l = [10, 1, 5, 2, 7 , 9]
# # list(reversed(np.argsort(l)[(np.sort(l) < 5).sum():]))
# np.argsort(l)[::-1]

In [10]:
from torch.utils.data import Sampler, SequentialSampler
from typing import Sized, Iterator
class MySampler(Sampler[int]):
    data_source: Sized
        
    def __init__(self, data_source: Sized) -> None:
        self.data_source = data_source
        self.mode = 'all'
        self.priority =  np.zeros(len(data_source))
#         self.threshold = 0
        self.curriculum = 0
    def __iter__(self) -> Iterator[int]:
        if self.mode == 'active':
#             return reversed(np.argsort(self.priority)[(np.sort(self.priority) < self.threshold).sum():])
            return reversed(np.argsort(self.priority)[::-1][:int(self.curriculum * len(self.data_source))])
        else:
            return iter(range(len(self.data_source)))
        
    def __len__(self) -> int:
#         return len(self.data_source) if self.mode == 'all' else (np.array(self.priority) > self.threshold).sum()
        return len(self.data_source) if self.mode == 'all' else int(self.curriculum * len(self.data_source))

    def set_priority(self, priority):
        self.priority = priority
        
#     def set_threshhold(self, threshold):
#         self.threshold = threshold
    def set_curriculum(self, curriculum):
        self.curriculum = curriculum
        
    def set_mode(self, mode):
        self.mode = mode
        
sampler = MySampler(train_dataset)
# list(MySampler([1, 2, 3, 10]))

In [11]:
# l[(np.sort(sampler.priority) < sampler.threshold).sum():]
# self.

In [12]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)#, collate_fn=data_collator)
eval_dataloader = DataLoader(val_dataset, batch_size)#, collate_fn=data_collator)

In [13]:
# train_dataloader.sampler.set_priority(list(range(len(train_dataset) - 1)) + [-5])
# for ind, batch in train_dataloader:
#     print(ind)
# {k: v.shape for k, v in batch.items()}

In [14]:
# list(sampler)

In [15]:

now = datetime.now()
full_name = f'{name}_{exec_number}_{repo.rsplit("/", 1)[1][-20:]}_{sample_percent}'
model_directory = f'{storage_directory}/tmp/finetuned/{full_name}'
model_directory

'./storage//tmp/finetuned/curr_336_qooxdoo_1'

In [16]:
tokenizer = T5Tokenizer.from_pretrained(load_model)
model = T5ForConditionalGeneration.from_pretrained(load_model)
model.resize_token_embeddings(len(tokenizer))
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32104, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32104, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [17]:
num_train_epochs = 370


In [18]:
# predictions = np.array(outputs[1].argmax(-1).to('cpu'))
# labels = np.array(batch['labels'].to('cpu'))
# np.sum(np.all(np.equal(predictions, labels), axis=1))

In [19]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wd)

In [20]:
from transformers import get_scheduler

num_training_steps = num_train_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=ws,
    num_training_steps=num_training_steps,
)
print(num_training_steps)


7770


In [21]:
import copy

In [22]:
# for index, batch in train_dataloader:
#     batch = {k: v.to('cuda') for k, v in batch.items()}
#     print(index)#, batch)
#     break

In [23]:
# [value.item() for key, value in sorted(list(zip(list(sampler)[:16], outputs[1].max(-1).values.sum(1))))]

In [24]:
# model.eval()
# scores = []
# for batch in train_dataloader:
#     batch = {k: v.to('cuda') for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = model(**batch)
#     scores += [item.item() for item in outputs[1].max(-1).values.mean(1).to('cpu')]
# scores    
# # # print(f'epoch #{epoch} | loss: {loss:.2f}, accuracy : {all_corrects/ all_cnt:.3f}')

In [25]:
from torch.nn.functional import softmax

# [item.item() for item in softmax(outputs[1], dim=-1).max(-1).values.prod(-1).to('cpu')]

In [26]:
# (outputs[1].argmax(-1) != 0).sum(-1) # length
# (batch['input_ids'] != 0).sum(1)

In [28]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_train_epochs * len(train_dataloader)))

model.train()
best_val_accuracy, best_val_loss = 0, 1
patience = 30
best_model = copy.deepcopy(model)
no_imp = 0

curriculum = 0
for epoch in range(num_train_epochs):
    if curriculum < 1:
        curriculum += 0.3
    sampler.curriculum = curriculum
    sampler.set_mode('all')
    if epoch == 0  or True:
        model.eval()
        scores = []
        for batch in train_dataloader:
            batch = {k: v.to('cuda') for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
    #         scores += [item.item() for item in outputs[1].max(-1).values.mean(1).to('cpu')]
            scores += [item.item() for item in softmax(outputs[1], dim=-1).max(-1).values.prod(-1).to('cpu')]
#             scores += list((-1 * outputs[1].argmax(-1).to('cpu') != 0).sum(-1)) # length of labels
#             scores += list((-1 * batch['input_ids'] != 0).sum(1).cpu())
        new_priorities = [value for key, value in sorted(list(zip(list(sampler), scores)))]    
        sampler.set_priority(new_priorities)
        sampler.set_mode('active')
        print(len(sampler))
#         print(list(sampler))
    
    model.train()
    all_corrects, all_cnt = 0, 0
    print('sm', sampler.curriculum, len(sampler), len(train_dataloader))
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]#outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
        predictions = np.array(outputs[1].argmax(-1).to('cpu'))
        labels = np.array(batch['labels'].to('cpu'))
        corrects = np.sum(np.all(np.equal(predictions, labels), axis=1))
        
        all_cnt += len(batch['labels'])
        all_corrects += corrects
        
#     print(f'epoch #{epoch} | loss: {loss:.2f}, accuracy : {all_corrects/ all_cnt:.3f}')    
    
    val_corrects, val_cnt = 0, 0
    for batch in eval_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        val_loss = outputs[0]
        predictions = np.array(outputs[1].argmax(-1).to('cpu'))
        labels = np.array(batch['labels'].to('cpu'))
        corrects = np.sum(np.all(np.equal(predictions, labels), axis=1))
        val_cnt += len(batch['labels'])
        val_corrects += corrects
    val_accuracy = val_corrects/ val_cnt
    print(f'epoch #{epoch} | tr_loss:{loss:.2f} tr_acc:{all_corrects/all_cnt} val_loss: {val_loss:.2f}, val_accuracy: {val_accuracy:.3f}')    
    print('---')
    
    
    if  val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        no_imp = 0
        best_model = copy.deepcopy(model)
        best_epoch = epoch
    else:
        no_imp += 1
    if no_imp >= patience:
        print(f'terminating... using {best_epoch}')
        break
    
    
    


96
sm 0.3 96 6
epoch #0 | tr_loss:0.00 tr_acc:0.96875 val_loss: 0.32, val_accuracy: 0.436
---
193
sm 0.6 193 13
epoch #1 | tr_loss:0.00 tr_acc:0.772020725388601 val_loss: 0.34, val_accuracy: 0.436
---
289
sm 0.8999999999999999 289 19
epoch #2 | tr_loss:0.00 tr_acc:0.5294117647058824 val_loss: 0.33, val_accuracy: 0.336
---
386
sm 1.2 386 25
epoch #3 | tr_loss:0.00 tr_acc:0.4503105590062112 val_loss: 0.40, val_accuracy: 0.273
---
386
sm 1.2 386 25
epoch #4 | tr_loss:0.16 tr_acc:0.45652173913043476 val_loss: 0.46, val_accuracy: 0.227
---
386
sm 1.2 386 25
epoch #5 | tr_loss:0.00 tr_acc:0.38198757763975155 val_loss: 0.51, val_accuracy: 0.264
---
386
sm 1.2 386 25
epoch #6 | tr_loss:0.02 tr_acc:0.36335403726708076 val_loss: 0.54, val_accuracy: 0.300
---
386
sm 1.2 386 25
epoch #7 | tr_loss:0.00 tr_acc:0.38509316770186336 val_loss: 0.49, val_accuracy: 0.273
---
386
sm 1.2 386 25
epoch #8 | tr_loss:0.00 tr_acc:0.38198757763975155 val_loss: 0.48, val_accuracy: 0.236
---
386
sm 1.2 386 25
epoch

In [29]:
all_corrects, all_cnt = 0, 0
best_model.eval()
for batch in eval_dataloader:
    batch = {k: v.to('cuda') for k, v in batch.items()}
    with torch.no_grad():
        outputs = best_model(**batch)
    loss = outputs[0]
    predictions = np.array(outputs[1].argmax(-1).to('cpu'))
    labels = np.array(batch['labels'].to('cpu'))
    corrects = np.sum(np.all(np.equal(predictions, labels), axis=1))

    all_cnt += len(batch['labels'])
    all_corrects += corrects
    
print(f'epoch #{epoch} | loss: {loss:.2f}, accuracy : {all_corrects/ all_cnt:.3f}')

epoch #30 | loss: 0.31, accuracy : 0.500


In [30]:
len(outputs)

4

In [31]:
# from sklearn.metrics import accuracy_score
# import numpy as np
# def compute_metrics(p):
#     target_max_length = 256
#     predictions, labels = p.predictions, p.label_ids
    
#     predictions = np.pad(predictions, ((0, 0), (0, target_max_length - predictions.shape[1])), mode="constant")
#     predictions = np.delete(predictions, 0, axis=1)
#     predictions = np.insert(predictions, target_max_length - 1, 0, axis=1)

    

#     labels = np.array(labels)
#     labels = np.pad(labels, ((0, 0), (0, target_max_length - labels.shape[1])), mode="constant")
#     labels = np.delete(labels, 0, axis=1)
#     labels = np.insert(labels, target_max_length - 1, 0, axis=1)
    

#     correct_counter = np.sum(np.all(np.equal(labels, predictions), axis=1))
#     return {'acc': int(correct_counter)}


In [32]:

training_args = Seq2SeqTrainingArguments(
    output_dir=model_directory,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=ws,
    weight_decay=wd,
    logging_dir=model_directory,
    logging_steps=100,
    do_eval=True,
    evaluation_strategy="epoch",
    learning_rate=lr,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=1,
    eval_accumulation_steps=1,  # set this lower, if testing or validation crashes
    disable_tqdm=False,
    predict_with_generate=True,  # never set this to false.
    seed=42,  # default value
)

from transformers import EarlyStoppingCallback


trainer = Seq2SeqTrainer(
    model=best_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    optimizers=[torch.optim.Adam(params=model.parameters(), lr=lr), None],
    tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

In [33]:
# from sklearn.metrics import accuracy_score
# import numpy as np
# def compute_metrics(p):
#     target_max_length = 256
#     predictions, labels = p.predictions, p.label_ids
    
#     predictions = np.pad(predictions, ((0, 0), (0, target_max_length - predictions.shape[1])), mode="constant")
#     predictions = np.delete(predictions, 0, axis=1)
#     predictions = np.insert(predictions, target_max_length - 1, 0, axis=1)

    

#     labels = np.array(labels)
#     labels = np.pad(labels, ((0, 0), (0, target_max_length - labels.shape[1])), mode="constant")
#     labels = np.delete(labels, 0, axis=1)
#     labels = np.insert(labels, target_max_length - 1, 0, axis=1)
    

#     correct_counter = np.sum(np.all(np.equal(labels, predictions), axis=1))
#     return {'acc': int(correct_counter)}


In [34]:
# from transformers import EarlyStoppingCallback

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     optimizers=[torch.optim.Adam(params=model.parameters(), lr=lr), None],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
# )

In [35]:

# start_training = datetime.now()

# trainer.train()

# end_training = datetime.now()



In [36]:

tuned_model_dir = f'{model_directory}/best'
# tuned_model_dir='/scratch/arminz/tmp/finetuned'
trainer.save_model(tuned_model_dir)


end_all = datetime.now()
# import csv
# with open('tuner_runtime.csv', 'a') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow([name, repo, len(train_dataset), len(val_dataset), base_model, start_all, start_training, end_training, end_all])

# In[78]:


In [37]:
if local:
    from numba import cuda
    device = cuda.get_current_device()
    device.reset()
#

In [38]:
result = os.system(f'python hf_transformers/tfix_testing.py --load-model {tuned_model_dir} -bs 16 --model-name t5-small -d repo-based-included -r {repo}')
print(result)

/home/armin/TFix/env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
21it [00:18,  1.41it/s]              

start time:  13:42:15
['no-invalid-this', 'no-throw-literal', 'no-new-wrappers', 'guard-for-in', 'no-new-object', 'comma-style', 'prefer-spread', 'no-caller', 'no-extra-bind', 'no-array-constructor', 'prefer-rest-params', 'generator-star-spacing', 'no-this-before-super', 'no-extend-native', 'no-undef', 'no-useless-escape', 'no-dupe-keys', 'no-console', 'no-constant-condition', 'no-duplicate-case', 'no-empty', 'no-extra-semi', 'no-redeclare', 'no-cond-assign', 'no-extra-boolean-cast', 'no-fallthrough', 'no-unreachable', 'valid-typeof', 'no-unsafe-finally', 'no-unused-vars', 'no-debugger', 'no-unsafe-negation', 'no-case-declarations', 'no-self-assign', 'no-process-exit', 'no-inner-declarations', 'for-direction', 'no-compare-neg-zero', 'no-sparse-arrays', 'no-func-assign', 'no-const-assign', 'no-global-assign', 'use-isnan', 'no-unused-labels', 'require-yield', 'getter-return', 'no-dupe-class-members', 'no-ex-assign', 'constructor-super', 'no-new-symbol', 'no-empty-pattern', 'no-class-assi

In [39]:
import shutil

shutil.rmtree(tuned_model_dir)
#